In [1]:
!pip install pyprind

In [4]:
import pyprind
import pandas as pd
import os

basepath = 'aclImdb'

labels = {'pos': 1, 'neg': 0}
pbar = pyprind.ProgBar(50000)
df = pd.DataFrame()
for s in ('test', 'train'):
    for l in ('pos', 'neg'):
        path = os.path.join(basepath, s, l)
        for file in sorted(os.listdir(path)):
            with open(os.path.join(path, file), 
                      'r', encoding='utf-8') as infile:
                txt = infile.read()
            df = df.append([[txt, labels[l]]], 
                           ignore_index=True)
            pbar.update()
df.columns = ['review', 'sentiment']

C:\Users\PC\AppData\Local\Temp\ipykernel_4780\2283992236.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append([[txt, labels[l]]],
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:45


In [5]:
df

,review,sentiment
0,I went and saw this movie last night after bei...,1
1,Actor turned director Bill Paxton follows up h...,1
2,As a recreational golfer with some knowledge o...,1
3,"I saw this film in a sneak preview, and it is ...",1
4,Bill Paxton has taken the true story of the 19...,1
...,...,...
49995,"Towards the end of the movie, I felt it was to...",0
49996,This is the kind of movie that my enemies cont...,0
49997,I saw 'Descent' last night at the Stockholm Fi...,0
49998,Some films that you pick up for a pound turn o...,0


In [8]:
df.to_csv('movie_data.csv', index = False, encoding = 'utf-8')

BoW (Bag of Word)

In [31]:
# CntVec : 단어의 순서는 신경안쓴다. 통틀어 몇번나왔는지
# 문맥파악 x, 단어의 등장 빈도 -> 분위기 파악 정도 (긍정적인 단어가 몇번 나왔는지, 전문용어가 몇번 나왔는지), 스팸메일분류
# 최초 언어모델

import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

sentence = ['The apple is delicious.', 'The banana is sweet.', 'One apple and two banana please.']
CntVec = CountVectorizer()
bag = CntVec.fit_transform(sentence)

In [32]:
CntVec.vocabulary_ # 문장 분해, 고유번호 부여, CntVec이 만든 사전

{'the': 8,
 'apple': 1,
 'is': 4,
 'delicious': 3,
 'banana': 2,
 'sweet': 7,
 'one': 5,
 'and': 0,
 'two': 9,
 'please': 6}

In [33]:
tmp = bag.toarray()

In [35]:
tmp

array([[0, 1, 0, 1, 1, 0, 0, 0, 1, 0],
       [0, 0, 1, 0, 1, 0, 0, 1, 1, 0],
       [1, 1, 1, 0, 0, 1, 1, 0, 0, 1]], dtype=int64)

In [ ]:
CntVec.transform(['How are you?']).toarray

In [34]:
tmp.shape # 등장하는 단어의 위치에 1 
          # CntVec : 단어의 순서는 신경안쓴다. 통틀어 몇번나왔는지
          # 문맥파악 x, 단어의 등장 빈도 -> 분위기 파악 정도 (긍정적인 단어가 몇번 나왔는지, 전문용어가 몇번 나왔는지), 스팸메일분류
          # 최초 언어모델

(3, 10)

In [37]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf = TfidfTransformer(use_idf = True, norm = 'l2', smooth_idf = True)

tfidf.fit_transform(tmp).toarray()

# 문장에서 단어의 중요도. 값이 클수록 중요한단어

array([[0.        , 0.45985353, 0.        , 0.60465213, 0.45985353,
        0.        , 0.        , 0.        , 0.45985353, 0.        ],
       [0.        , 0.        , 0.45985353, 0.        , 0.45985353,
        0.        , 0.        , 0.60465213, 0.45985353, 0.        ],
       [0.44036207, 0.3349067 , 0.3349067 , 0.        , 0.        ,
        0.44036207, 0.44036207, 0.        , 0.        , 0.44036207]])

In [44]:
X_train = df.loc[:25000, 'review'].values # 앞 25000개가 train data
y_train = df.loc[:25000, 'sentiment'].values 
X_test = df.loc[25000:, 'review'].values
y_test = df.loc[25000:, 'sentiment'].values 

In [47]:
from sklearn.feature_extraction.text import TfidfVectorizer # count vectorizer와 tfidf transfomer를 합쳐놓은 것
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

tfidf = TfidfVectorizer(strip_accents = None, lowercase = False, preprocessor = None, stop_words = None, ngram_range = (1, 1)) 
# strip_accents 악센트, lowercase 대소문자구분, preprocessor 전처리, ngram_range 띄어쓰기 몇개를 한 단어로 볼 것인지

# 분류기와 Vectorizer를 Pipeline으로
tfidf_lr = Pipeline([('tfidf', tfidf), ('lr', LogisticRegression(solver = 'liblinear', C = 10, penalty = 'l2', random_state = 0))])  

In [48]:
tfidf_lr.fit(X_train, y_train)

Pipeline(steps=[('tfidf', TfidfVectorizer(lowercase=False)),
                ('lr',
                 LogisticRegression(C=10, random_state=0, solver='liblinear'))])

In [50]:
p_test = tfidf_lr.predict(X_test) # 긍정, 부정 리뷰 분류
p_test

array([1, 1, 1, ..., 0, 0, 0], dtype=int64)

In [53]:
# 점수 확인

from sklearn.metrics import f1_score, confusion_matrix, accuracy_score

print(accuracy_score(y_test, p_test)) # 0.8797...
print(f1_score(y_test, p_test)) # 0.8785.. 
print(confusion_matrix(y_test, p_test))

0.87972
0.878549214427077
[[11117  1383]
 [ 1624 10876]]


In [ ]:
가장 고전적인 모델